In [108]:
import pandas as pd
import numpy as np

Examples and info found here: https://info.arxiv.org/help/api/user-manual.html#detailed_examples 

In [20]:



#arXiv API example

import urllib, urllib.request
url = 'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=50'
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))


<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3Aelectron%26id_list%3D%26start%3D0%26max_results%3D50" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:electron&amp;id_list=&amp;start=0&amp;max_results=50</title>
  <id>http://arxiv.org/api/alEVQR3uLgIyLntrUTDVbGClTxc</id>
  <updated>2024-02-02T00:00:00-05:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">205944</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">50</opensearch:itemsPerPage>
  <entry>
    <id>http://arxiv.org/abs/cond-mat/0102536v1</id>
    <updated>2001-02-28T20:12:09Z</updated>
    <published>2001-02-28T20:12:09Z</published>
    <title>Impact of Electron-Electron Cusp on Confi

In [3]:
# psuedo code


# while data size < 500 mb
# start with one article
# search for each author on the paper, gathering metadata: 
    # date, authors, title, abstract, pdf link, etc. Keep unique id as DOI so papers can not be used twice
        # if DOI is in data, skip to next.
    # tree outward by author from starting article to gather top X articles by that author repeating the same process 
    


In [4]:
namespace= '{http://www.w3.org/2005/Atom}'
example = namespace + 'id'
example

'{http://www.w3.org/2005/Atom}id'

In [129]:
import time
import requests
import xml.etree.ElementTree as ET
import sys
import io
import PyPDF2
from pympler import asizeof




class arXiv_crawler:
    def __init__(self, data_size, starting_query):
        #self.starting_keyword = starting_keyword
        self.data_size = data_size  #data size 
        self.articles = {} #empty dict for data
        self.namespace = '{http://www.w3.org/2005/Atom}' #neede to parse XML
        self.starting_query = starting_query
        self.pdf_counter = 0 #keep track of pdfs, for logging
    
    
    def starting_article(self):
        #get the starting article and parse
        url = f'http://export.arxiv.org/api/query?search_query=all:{self.starting_query}&start=0&max_results=1'
        self.parse_XML(url)
        
    def parse_XML(self, url):
        response = requests.get(url) #fetch data from link
        
        #added incase errors
        if response.status_code != 200:
            print(f"Error fetching data HTTP {response.status_code}")
            return
        
        #handle and catcches errors 
        try:
            root =  ET.fromstring(response.content)

            for entry in root.findall(self.namespace + 'entry'): #parse and process each entry
                #if ID is not in self.articles['id']:

                article_id = entry.find(self.namespace + 'id').text

                #create dict for data
                article_data = {
                'id':article_id,
                'title': entry.find(self.namespace + 'title').text,
                'summary': entry.find(self.namespace + 'summary').text.strip(),
                'published': entry.find(self.namespace + 'published').text,
                'authors': [author.find(self.namespace + 'name').text for author in entry.findall(self.namespace +'author')]}

                #get the pdf link to article
                for link in entry.findall(self.namespace + 'link'):
                    if link.get('type') == 'application/pdf':
                        article_data['pdf_link'] = link.get('href')
                        break
                #print(f"pdf link {article_data['pdf_link']}")

                #extract pdf text
                article_data['pdf_text'] = self.pdf_to_text(article_data['pdf_link'])

                #print(f"Completed extracting text from: {article_data['title']}")

                #avoid duplicat
                if article_id not in self.articles:
                    self.articles[article_id] = article_data
        
        except ET.ParseError as e:
            print(f"Parse error: {e}")
                
        #return self.articles
    
    def pdf_to_text(self, pdf_url):
        #handle exceptions
        try:
            response = requests.get(pdf_url) #fetch data from pdf url
            self.pdf_counter +=1  #keep track of pdfs, for logging
            
            #
            if response.status_code == 200: #if successful, process pdf
                
                pdf_file = io.BytesIO(response.content) 
                reader = PyPDF2.PdfReader(pdf_file) 
                num_pages = len(reader.pages) #number of pages in pdf
                content = [] #store content in list (each page)
                
                #iterate thru each page, extracting text
                for page in range(num_pages):
                    content.append(reader.pages[page].extract_text()) 
                print(f'successfully extracted text from PDF: {pdf_url}') #logging statement
                
                #for my own sanity to check progress
                if self.pdf_counter % 50 == 0:
                    print(f"current size {self.check_size()} mb, after {self.pdf_counter} articles")
                    
                return content
            
            else:
                return f"Failed to dl pdf, error code {response.status_code}" #else return error code
        
        except Exception as e:
            return f"error processing pdf: {e}"
        
        
    def check_size(self):
        return asizeof.asizeof(self.articles) / (1024 ** 2)   #convert to mb

    def search_authors(self, authors, max_results=100):
        searched_authors = set()  #avoid repeat author searched
        
        #continue looping until stopping condition is met
        while authors:
            
            author = authors.pop() #removes top author 
            
            #if author has been searched, continue
            if author in searched_authors: 
                continue
                
            searched_authors.add(author) #add author to searched
            
            #query the author with up to 100 of their papers
            query_url = f"http://export.arxiv.org/api/query?search_query=au:{author}&start=0&max_results={max_results}"
            print(f"Fetching articles for author: {author}")  #prnit log
            self.parse_XML(query_url) #parse the url
            
            current_size = self.check_size() #check the size
            print(f"Current data size after author {author}: {current_size} mb")
            
            #stopping statement
            if current_size >= self.data_size:
                print("mem limit hit, stopping crawl.")
                return
             #"In cases where the API needs to be called multiple times in a row, we encourage you to play nice and incorporate a 3 second delay in your code."
            time.sleep(3)
            
            #iterates over the articles stored in articles 
            for article in self.articles.values():
                #create a co-author set for each article
                article_authors_set = set(article['authors']) 
                
                #ensures all future author searches are unique (not in searched authors)
                authors.update(article_authors_set.difference(searched_authors))


    def crawl(self):
        print("Starting arXiv crawl!")
        self.starting_article()
        current_size = self.check_size()
        print(f"Initial data size: {current_size} MB")


        authors = set()
        for article in self.articles.values():
            authors.update(article['authors'])
                
        print(f"Searching for articles by {len(authors)} authors")
        self.search_authors(authors)
            
        final_size = self.check_size()
        print(f"Final data size: {final_size} mb")

        return self.articles #return article dictionary

In [130]:
crawler = arXiv_crawler(data_size=1200, starting_query='electron')
data = crawler.crawl()

Starting arXiv crawl!
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0102536v1
Initial data size: 0.08750152587890625 MB
Searching for articles by 5 authors
Fetching articles for author: J. C. Greer
successfully extracted text from PDF: http://arxiv.org/pdf/0710.5276v1
successfully extracted text from PDF: http://arxiv.org/pdf/0912.3431v1
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0312522v1
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0105367v1
successfully extracted text from PDF: http://arxiv.org/pdf/1002.0719v1
successfully extracted text from PDF: http://arxiv.org/pdf/1609.07624v1
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0301241v1
successfully extracted text from PDF: http://arxiv.org/pdf/1707.02045v2
successfully extracted text from PDF: http://arxiv.org/pdf/0804.0868v1
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0102536v1
successfully extracted text from PDF: ht

unknown widths : 
[0, IndirectObject(5186, 0, 2272239360448)]
unknown widths : 
[0, IndirectObject(5188, 0, 2272239360448)]
unknown widths : 
[0, IndirectObject(5190, 0, 2272239360448)]
unknown widths : 
[0, IndirectObject(5192, 0, 2272239360448)]
unknown widths : 
[0, IndirectObject(5194, 0, 2272239360448)]
unknown widths : 
[0, IndirectObject(5196, 0, 2272239360448)]


successfully extracted text from PDF: http://arxiv.org/pdf/2002.02967v3
successfully extracted text from PDF: http://arxiv.org/pdf/2002.03005v2


unknown widths : 
[0, IndirectObject(2291, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(2293, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3133, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3135, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3137, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3139, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3141, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3143, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3461, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3463, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3465, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3467, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3469, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3471, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3473, 0, 2273513682736)]
unknown widths : 
[0, IndirectObject(3475, 0, 2273513682736)]
unknown 

successfully extracted text from PDF: http://arxiv.org/pdf/2002.03008v3


...

successfully extracted text from PDF: http://arxiv.org/pdf/1512.01032v1
successfully extracted text from PDF: http://arxiv.org/pdf/1008.2296v1
successfully extracted text from PDF: http://arxiv.org/pdf/1608.04033v2
successfully extracted text from PDF: http://arxiv.org/pdf/1508.02381v1
successfully extracted text from PDF: http://arxiv.org/pdf/1804.00591v3
successfully extracted text from PDF: http://arxiv.org/pdf/2001.06060v1
successfully extracted text from PDF: http://arxiv.org/pdf/1512.06043v2
successfully extracted text from PDF: http://arxiv.org/pdf/1603.05040v1
successfully extracted text from PDF: http://arxiv.org/pdf/1509.04296v2
successfully extracted text from PDF: http://arxiv.org/pdf/1709.05689v1
successfully extracted text from PDF: http://arxiv.org/pdf/1507.05137v3
successfully extracted text from PDF: http://arxiv.org/pdf/1505.01871v2
current size 1172.523910522461 mb, after 14600 articles
successfully extracted text from PDF: http://arxiv.org/pdf/1611.03184v1
successfu

successfully extracted text from PDF: http://arxiv.org/pdf/hep-th/0304007v2
current size 1178.7066268920898 mb, after 14700 articles
successfully extracted text from PDF: http://arxiv.org/pdf/math/0203055v1
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0603468v1
successfully extracted text from PDF: http://arxiv.org/pdf/cond-mat/0603808v1
successfully extracted text from PDF: http://arxiv.org/pdf/1311.4283v3
successfully extracted text from PDF: http://arxiv.org/pdf/2308.02722v1
successfully extracted text from PDF: http://arxiv.org/pdf/astro-ph/0509080v2
successfully extracted text from PDF: http://arxiv.org/pdf/1706.08157v1
successfully extracted text from PDF: http://arxiv.org/pdf/math/0504192v2
Current data size after author I. Y. Park: 1179.1901626586914 mb
Fetching articles for author: Q. T. Lei
successfully extracted text from PDF: http://arxiv.org/pdf/1909.03225v1
successfully extracted text from PDF: http://arxiv.org/pdf/1004.2682v1
successfully extracted

Current data size after author Q. T. Lei: 1184.6292266845703 mb
Fetching articles for author: M. M. Chowdhury
successfully extracted text from PDF: http://arxiv.org/pdf/0708.2395v2
successfully extracted text from PDF: http://arxiv.org/pdf/cs/0611065v3
successfully extracted text from PDF: http://arxiv.org/pdf/0708.2571v5
successfully extracted text from PDF: http://arxiv.org/pdf/0708.2397v5
successfully extracted text from PDF: http://arxiv.org/pdf/1405.5169v1
successfully extracted text from PDF: http://arxiv.org/pdf/1812.06370v2
successfully extracted text from PDF: http://arxiv.org/pdf/2006.02113v1
successfully extracted text from PDF: http://arxiv.org/pdf/1812.05758v1
successfully extracted text from PDF: http://arxiv.org/pdf/2101.09541v1
successfully extracted text from PDF: http://arxiv.org/pdf/2204.04736v1
successfully extracted text from PDF: http://arxiv.org/pdf/1706.05631v1
successfully extracted text from PDF: http://arxiv.org/pdf/1706.05634v1
successfully extracted text fr

successfully extracted text from PDF: http://arxiv.org/pdf/2006.07492v1
successfully extracted text from PDF: http://arxiv.org/pdf/astro-ph/0305304v1
successfully extracted text from PDF: http://arxiv.org/pdf/2107.02238v2
successfully extracted text from PDF: http://arxiv.org/pdf/1307.4762v1
successfully extracted text from PDF: http://arxiv.org/pdf/math/0609240v2
successfully extracted text from PDF: http://arxiv.org/pdf/2201.09692v1
successfully extracted text from PDF: http://arxiv.org/pdf/astro-ph/0001384v1
successfully extracted text from PDF: http://arxiv.org/pdf/hep-th/9910223v1
successfully extracted text from PDF: http://arxiv.org/pdf/1505.08161v1
successfully extracted text from PDF: http://arxiv.org/pdf/2311.09578v1
successfully extracted text from PDF: http://arxiv.org/pdf/1601.01404v1
successfully extracted text from PDF: http://arxiv.org/pdf/1901.00866v3
successfully extracted text from PDF: http://arxiv.org/pdf/2111.13786v1
successfully extracted text from PDF: http://ar

successfully extracted text from PDF: http://arxiv.org/pdf/1307.4052v2
successfully extracted text from PDF: http://arxiv.org/pdf/1310.2584v1
successfully extracted text from PDF: http://arxiv.org/pdf/1508.05741v2
successfully extracted text from PDF: http://arxiv.org/pdf/1508.06085v1
successfully extracted text from PDF: http://arxiv.org/pdf/1011.5897v3
successfully extracted text from PDF: http://arxiv.org/pdf/1503.08452v3
successfully extracted text from PDF: http://arxiv.org/pdf/1706.09459v3
successfully extracted text from PDF: http://arxiv.org/pdf/1905.05638v1
successfully extracted text from PDF: http://arxiv.org/pdf/1905.06702v1
successfully extracted text from PDF: http://arxiv.org/pdf/1905.13274v1
successfully extracted text from PDF: http://arxiv.org/pdf/1608.01718v1
successfully extracted text from PDF: http://arxiv.org/pdf/1909.13314v1
successfully extracted text from PDF: http://arxiv.org/pdf/1811.06076v1
successfully extracted text from PDF: http://arxiv.org/pdf/1906.030

In [79]:
!pip install pympler

     -------------------------------------- 164.8/164.8 kB 2.5 MB/s eta 0:00:00


In [23]:
#json_test = json.dumps(data)
#print(json_test)

In [131]:
articles_list = [data[article_id] for article_id in data]
df = pd.DataFrame(articles_list)
df

,id,title,summary,published,authors,pdf_link,pdf_text
0,http://arxiv.org/abs/cond-mat/0102536v1,Impact of Electron-Electron Cusp on Configurat...,The effect of the electron-electron cusp on th...,2001-02-28T20:12:09Z,"[David Prendergast, M. Nolan, Claudia Filippi,...",http://arxiv.org/pdf/cond-mat/0102536v1,[arXiv:cond-mat/0102536v1 [cond-mat.str-el] ...
1,http://arxiv.org/abs/0710.5276v1,Electronegativity in quantum electronic transport,Electronegativity is shown to control charge t...,2007-10-28T11:27:36Z,"[R. J. Bartlett, G. Fagas, J. C. Greer]",http://arxiv.org/pdf/0710.5276v1,[arXiv:0710.5276v1 [cond-mat.mes-hall] 28 Oc...
2,http://arxiv.org/abs/0912.3431v1,"Comment on ""Electron transport through correla...",The many electron correlated scattering (MECS)...,2009-12-17T18:05:36Z,"[J. C. Greer, P. Delaney, G. Fagas]",http://arxiv.org/pdf/0912.3431v1,[Comment on \Electron transport through correl...
3,http://arxiv.org/abs/cond-mat/0312522v1,Correlated Electron Transport,Theoretical and experimental values to date fo...,2003-12-19T11:56:11Z,"[P. Delaney, J. C. Greer]",http://arxiv.org/pdf/cond-mat/0312522v1,[arXiv:cond-mat/0312522v1 19 Dec 2003Correlat...
4,http://arxiv.org/abs/cond-mat/0105367v1,Many-Electron Systems with Constrained Current,A formulation for transport in an inhomogeneou...,2001-05-18T09:02:21Z,"[D. S. Kosov, J. C. Greer]",http://arxiv.org/pdf/cond-mat/0105367v1,[arXiv:cond-mat/0105367v1 18 May 2001Many-Ele...
...,...,...,...,...,...,...,...
13379,http://arxiv.org/abs/1701.04875v1,Nowhere differentiable functions with respect ...,Let $\Omega$ be a bounded domain in $\mathbb{C...,2017-01-17T21:32:53Z,"[K. Kavvadias, K. Makridis]",http://arxiv.org/pdf/1701.04875v1,[arXiv:1701.04875v1 [math.CV] 17 Jan 2017Now...
13380,http://arxiv.org/abs/1510.00194v2,Nonlocal dynamics and infinite non-relativisti...,We study the symmetry of the class of nonlocal...,2015-10-01T11:57:16Z,"[K. Andrzejewski, K. Bolonek-Lason]",http://arxiv.org/pdf/1510.00194v2,[arXiv:1510.00194v2 [hep-th] 22 Dec 2015Nonl...
13381,http://arxiv.org/abs/0808.2535v1,High Kondo temperature (TK ~ 80 K) in self-ass...,We have fabricated single electron tunneling s...,2008-08-19T06:09:33Z,"[K. Shibata, K. Hirakawa]",http://arxiv.org/pdf/0808.2535v1,[ 1High Kondo temperature ( TK ~ 80 K) in self...
13382,http://arxiv.org/abs/1210.1503v2,Semiclassical expansion of the Slater sum for ...,We consider hamiltonian systems with spatially...,2012-10-04T16:07:22Z,"[K. Berkane, K. Bencheikh]",http://arxiv.org/pdf/1210.1503v2,[arXiv:1210.1503v2 [quant-ph] 26 Apr 2013Sem...


In [132]:
df.to_csv('Full_arXiv_data.csv')

In [138]:
# no duplicates
len(df['id'].unique()) == df.shape[0]

True

In [139]:
df['pdf_text'][12473]

['Infrared Behavior of Three-Point Functions in\nLandau Gauge Yang-Mills Theory\nMarkus Huber\x03\nKarl-Franzens University Graz\nE-mail: markus.huber@uni-graz.at\nReinhard Alkofer\nKarl-Franzens University Graz\nE-mail: reinhard.alkofer@uni-graz.at\nKai Schwenzer\nKarl-Franzens University Graz\nE-mail: kai.schwenzer@uni-graz.at\nThe three-gluon and ghost-gluon vertices of Landau gauge Yang-Mills theory are investigated in\nthe low momentum regime. Due to ghost dominance in the infrared we can use the known power\nlaw behavior for the propagators to determine analytically the complete momentum dependence\nof the dressing functions. Besides a uniform, i. e. all momenta going to zero, divergence, we\nﬁnd additional singularities, if one momentum alone goes to zero, while the other two remain\nconstant. At these asymmetric points we can extract additional infrared exponents, which cor-\nroborate previous results and expand the known ﬁxed point solution of Landau gauge Yang-Mills\ntheory, 

In [95]:
df['pdf_text'][0]

['arXiv:cond-mat/0102536v1  [cond-mat.str-el]  28 Feb 2001Impact of Electron-Electron Cusp on Conﬁguration Interact ion Energies\nDavid Prendergast,a)M. Nolan,b)Claudia Filippi,a)Stephen Fahy,a)and J.C. Greerb)\na)Department of Physics andb)NMRC, University College, Cork, Ireland\n(February 1, 2008)\nThe eﬀect of the electron-electron cusp on the convergence o f conﬁguration interaction (CI) wave\nfunctions is examined. By analogy with the pseudopotential approach for electron-ion interactions,\nan eﬀective electron-electron interaction is developed wh ich closely reproduces the scattering of\nthe Coulomb interaction but is smooth and ﬁnite at zero elect ron-electron separation. The exact\nmany-electron wave function for this smooth eﬀective inter action has no cusp at zero electron-\nelectron separation. We perform CI and quantum Monte Carlo c alculations for He and Be atoms,\nboth with the Coulomb electron-electron interaction and wi th the smooth eﬀective electron-electron\ninteract

In [147]:
df.shape

(13384, 7)